In [1]:
import os
import numpy as np
import pandas as pd

In [14]:
sample_id = 'HG00533'

online_candidate_res_root_dir=r'/zfssz2/ST_MCHRI/BIGDATA/PROJECT/NIPT_CNV/f_cnv_out/online/1KGP_bak/'+sample_id+'/cnv_call'
online_re_fname = 'Analysis{}_1000_200_out_cnv.csv'.format(sample_id)
online_fn = os.path.join(online_candidate_res_root_dir, online_re_fname)

in_out_pred = pd.read_csv(online_fn, sep='\t')
in_out_pred.head()

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,...,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,CHROM,TRUE_L,CENTOR_INTER_LEN,SUPDUP_INTER_LEN,CENT_CALLLEN_RATIO,SUPDUP_CALLLEN_RATIO
0,151400,151600,200,0.116070,0.883346,0.000584,1,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
1,164600,164800,200,0.270108,0.011905,0.717987,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
2,166600,167600,1000,0.217168,0.397399,0.385434,1,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
3,362168,362968,800,0.396992,0.201133,0.401874,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
4,375568,376568,1000,0.413482,0.004589,0.581928,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0


In [15]:
gtruth_cnv_base_dir = '/zfssz6/ST_MCHRI/BIGDATA/P18Z10200N0124/NIPT_CNV/NIPT_CNV/1k_cnvs_lab_feat'
gtruth_fn = os.path.join(gtruth_cnv_base_dir, 'ALL.wgs.mergedSV.v8.20130502.svs.genotypes.GRCh38.vcf.' +
        sample_id + '.cnvs.labels')
gtruth_cnv_pd = pd.read_csv(gtruth_fn, sep='\t')
# gtruth_cnv_pd = gtruth_cnv_pd[gtruth_cnv_pd['CHROM']==chr_id]
print('the total number of cnvs for sample {} is {}'.format(sample_id, gtruth_cnv_pd.shape[0]))

true_del_df = gtruth_cnv_pd[gtruth_cnv_pd['ADJTYPE']=='DEL']
true_del_len_arr = true_del_df['SVLEN'].values
print('>'* 4 +'where DEL is {}, length range {}-{}, median: {}'.format(
    true_del_df.shape[0], min(true_del_len_arr), max(true_del_len_arr), np.median(true_del_len_arr)))
true_dup_df = gtruth_cnv_pd[gtruth_cnv_pd['ADJTYPE']=='DUP']
true_dup_len_arr = true_dup_df['SVLEN'].values
print('>'* 4 +'where DUP is {}, length range {}-{}, median: {}'.format(
    true_dup_df.shape[0], min(true_dup_len_arr), max(true_dup_len_arr), np.median(true_dup_len_arr)))

# check truth cnv labels with unpredective region
true_unpredictive_df = in_out_pred[(in_out_pred['PRED_L']==-1) & (~in_out_pred['ID'].isnull())]
uppred_true_lab_lst = np.unique(true_unpredictive_df['ID'].values)
# check all truth cnv labels
true_cnv_df = in_out_pred[(in_out_pred['PRED_L']!=-1) &(~in_out_pred['ID'].isnull())]
pred_true_lab_lst = np.unique(true_cnv_df['ID'].values)

unpred_cnvs = set(uppred_true_lab_lst) - set(pred_true_lab_lst)
unpred_dict = dict()
unpred_dict['DEL']=[]
unpred_dict['DUP']=[]
for i in unpred_cnvs:
    i_lbl = gtruth_cnv_pd.loc[gtruth_cnv_pd['ID']==i, 'ADJTYPE'].values[0]
    unpred_dict[i_lbl].append(i)
print('CNVs that are located at non-read region: ',unpred_dict)

the total number of cnvs for sample HG00533 is 1631
>>>>where DEL is 1506, length range 50-270689, median: 618.5
>>>>where DUP is 125, length range 3083-160657, median: 26455.0
CNVs that are located at non-read region:  {'DEL': [], 'DUP': []}


In [72]:
out_pred=in_out_pred[in_out_pred['LEN']>=1000]
# out_pred = in_out_pred

In [73]:
out_pred.shape

(41685, 26)

In [74]:
out_pred.loc[(out_pred['P_DUP'] < 0.8 ) &
                    (out_pred['PRED_L'] == 2), 'PRED_L'] = 0
out_pred.loc[(out_pred['P_DEL'] < 0.8) &
                    (out_pred['PRED_L'] == 1), 'PRED_L'] = 0

/zfssz2/ST_MCHRI/BIGDATA/USER/baiyong/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [75]:
out_pred.loc[(np.abs(out_pred['P_DUP'] - out_pred['P_NEU']) < 0.1) &
                    (out_pred['PRED_L'] == 2), 'PRED_L'] = 0
out_pred.loc[(np.abs(out_pred['P_DEL'] - out_pred['P_NEU']) < 0.1) &
                    (out_pred['PRED_L'] == 1), 'PRED_L'] = 0
    # 2 reduce fp
tmp_df = out_pred[['P_NEU', 'P_DEL', 'P_DUP']]
tmp_df.columns = ['0', '1', '2']
out_pred['PRED_P_L'] = tmp_df.idxmax(axis=1, skipna=True)
out_pred['PRED_P_L'] = out_pred['PRED_P_L'].astype(int)
out_pred.loc[(out_pred['PRED_L'] != 0) & (out_pred['PRED_P_L'] == 0), 'PRED_L'] = 0

/zfssz2/ST_MCHRI/BIGDATA/USER/baiyong/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/zfssz2/ST_MCHRI/BIGDATA/USER/baiyong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/zfssz2/ST_MCHRI/BIGDATA/USER/baiyong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [76]:
true_cnvs = out_pred.loc[out_pred['TRUE_L']!=0,
                         ['ID', 'TRUE_L']].drop_duplicates( 
                     keep = 'first', inplace = False) 

In [77]:
true_dup=true_cnvs[true_cnvs['TRUE_L']==2]
true_del=true_cnvs[true_cnvs['TRUE_L']==1]

In [78]:
 true_del.shape, true_dup.shape

((540, 2), (114, 2))

In [79]:
false_dup_df = out_pred[(out_pred['PRED_L']==2) & (out_pred['ID'].isnull())]
false_del_df = out_pred[(out_pred['PRED_L']==1) & (out_pred['ID'].isnull())]
false_neg_del_df = out_pred[(out_pred['PRED_L']==0) & (out_pred['TRUE_L']==1)]
false_neg_dup_df = out_pred[(out_pred['PRED_L']==0) & (out_pred['TRUE_L']==2)]
tp_del_df = out_pred[(out_pred['PRED_L']==1) & (out_pred['TRUE_L']==1)]
tp_dup_df = out_pred[(out_pred['PRED_L']==2) & (out_pred['TRUE_L']==2)]

In [80]:
false_dup_df.shape, false_del_df.shape, false_neg_del_df.shape, false_neg_dup_df.shape

((108, 26), (407, 26), (819, 26), (948, 26))

In [81]:
tp_del_df.shape, tp_dup_df.shape

((56, 26), (36, 26))

In [83]:
90/650

0.13846153846153847

In [64]:
false_del_df.head(30)

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,...,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,CHROM,TRUE_L,CENTOR_INTER_LEN,SUPDUP_INTER_LEN,CENT_CALLLEN_RATIO,SUPDUP_CALLLEN_RATIO,PRED_P_L
184,6304290,6305490,1200,0.003235,0.996545,0.000220,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
541,15454984,15455984,1000,0.040868,0.955109,0.004023,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
810,22001363,22002363,1000,0.009695,0.969355,0.020950,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
1505,39233835,39234835,1000,0.007488,0.969156,0.023356,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
2178,52850035,52851035,1000,0.027953,0.972020,0.000027,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
2923,70368035,70369035,1000,0.028322,0.971642,0.000036,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
5073,112810635,112811635,1000,0.032546,0.957627,0.009827,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
5616,121585635,121586635,1000,0.042876,0.957091,0.000033,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
5711,143631787,143632987,1200,0.039947,0.958318,0.001735,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
7603,170598787,170599787,1000,0.027567,0.971443,0.000991,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
